In [9]:
from transformers import pipeline

# Utiliser un pipeline de classification zéro-shot
classifier = pipeline("zero-shot-classification", model="mtheo/camembert-base-xnli")
# classifier = pipeline("zero-shot-classification", model="cmarkea/distilcamembert-base-nli")

# Fonction pour diviser le document en segments
def split_into_segments(text, max_length=512):
    words = text.split()
    segments = []
    current_segment = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1
        if current_length > max_length:
            segments.append(" ".join(current_segment))
            current_segment = []
            current_length = len(word) + 1
        current_segment.append(word)

    if current_segment:
        segments.append(" ".join(current_segment))

    return segments

# Documents à classifier
documents = [
    """La glace s’accumule des que les pr ` ecipitations neigeuses d ´ epassent la fonte estivale et l’ ´ ecoulement. ´ La position des calottes depend donc de la proximit ´ e des zones oc ´ eaniques relativement chaudes ´ (evaporation importante et transport direct de la vapeur vers les zones froides : refroidissement ´ local important) Pour etudier les enregistrements : besoin de carottes de glace ´ Exemple de la calotte antarctique (forages EPICA, Vostok, Dome C et F,...) Important de pouvoir dater la glace des carottes : – Reconnaissance des couches annuelles – Identification des annees de r ´ ef´ erences (ex. : volcanisme) ´ – Modelisation des variations d’accumulation, de l’ ´ ecoulement de la glace ´ – Correlations entre les diff ´ erents forages (CO ´ 2, CH4) et les sediments marins ( ´ δ 18O de O2 atmospherique).""",
]

# Catégories à prédire
categories = [
    "Enregistrements des paléoclimats et des phénomènes tectoniques",
    "La cellule, unité structurale et fonctionnelle du vivant",
    "L’organisme, une société de cellules",
    "Plans d’organisation du vivant et phylogénie"
]

# Classification des documents
for doc in documents:
    segments = split_into_segments(doc, max_length=512)
    aggregated_results = {category: 0 for category in categories}

    for segment in segments:
        results = classifier(segment, candidate_labels=categories)
        for label, score in zip(results['labels'], results['scores']):
            aggregated_results[label] += score

    # Normaliser les scores
    total_score = sum(aggregated_results.values())
    normalized_results = {label: score / total_score for label, score in aggregated_results.items()}

    print(f"Document: {doc[:100]}...")  # Affiche une partie du document pour un aperçu
    print(f"Predicted categories: {normalized_results}")


Document: La théorie astronomique des paléoclimats est un modèle utilisé en sciences de la Terre pour explique...
Predicted categories: {'Enregistrements des paléoclimats et des phénomènes tectoniques': 0.3226195174845981, 'La cellule, unité structurale et fonctionnelle du vivant': 0.20847179802961843, 'L’organisme, une société de cellules': 0.14542173562578958, 'Plans d’organisation du vivant et phylogénie': 0.32348694885999385}
